In [1]:
#hide
#default_exp sync
#default_cls_lvl 3
from nbdev.showdoc import show_doc

In [2]:
#export
from nbdev.imports import *
# from nbdev.export import *
from fastcore.script import *
import nbformat
from nbformat.sign import NotebookNotary

In [3]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.getcwd()) + '/nbdev'
sys.path.append(os.path.normpath(SCRIPT_DIR))
from export_scala import *

# Synchronize and diff

> The functions that propagates small changes in the library back to notebooks



The library is primarily developed in notebooks so any big changes should be made there. But sometimes, it's easier to fix small bugs or typos in the modules directly. `nbdev_update_lib` is the function that will propagate those changes back to the corresponding notebooks. Note that you can't create new cells with that functionality, so your corrections should remain limited.

## Finding the way back to notebooks

We need to get the name of the object we are looking for, and then we'll try to find it in our index file.

In [4]:
#export
def _get_property_name(p):
    "Get the name of property `p`"
    if hasattr(p, 'fget'):
        return p.fget.func.__qualname__ if hasattr(p.fget, 'func') else p.fget.__qualname__
    else: return next(iter(re.findall(r'\'(.*)\'', str(p)))).split('.')[-1]

def get_name(obj):
    "Get the name of `obj`"
    if hasattr(obj, '__name__'):       return obj.__name__
    elif getattr(obj, '_name', False): return obj._name
    elif hasattr(obj,'__origin__'):    return str(obj.__origin__).split('.')[-1] #for types
    elif type(obj)==property:          return _get_property_name(obj)
    else:                              return str(obj).split('.')[-1]

In [5]:
from nbdev.export import DocsTestClass

In [6]:
test_eq(get_name(in_ipython), 'in_ipython')
test_eq(get_name(DocsTestClass.test), 'test')

In [7]:
#export
def qual_name(obj):
    "Get the qualified name of `obj`"
    if hasattr(obj,'__qualname__'): return obj.__qualname__
    if inspect.ismethod(obj):       return f"{get_name(obj.__self__)}.{get_name(fn)}"
    return get_name(obj)

In [8]:
test_eq(qual_name(DocsTestClass.test), 'DocsTestClass.test')

In [9]:
#export
def source_nb(func, is_name=None, return_all=False, mod=None):
    "Return the name of the notebook where `func` was defined"
    is_name = is_name or isinstance(func, str)
    if mod is None: mod = get_nbdev_module()
    index = mod.index
    name = func if is_name else qual_name(func)
    while len(name) > 0:
        if name in index: return (name,index[name]) if return_all else index[name]
        name = '.'.join(name.split('.')[:-1])

You can either pass an object or its name (by default `is_name` will look if `func` is a string or not to determine if it should be `True` or `False`, but you can override if there is some inconsistent behavior). 

If passed a method of a class, the function will return the notebook in which the largest part of the function name was defined in case there is a monkey-matching that defines `class.method` in a different notebook than `class`. If `return_all=True`, the function will return a tuple with the name by which the function was found and the notebook.

For properties defined using `property` or our own `add_props` helper, we approximate the name by looking at their getter functions, since we don't seem to have access to the property name itself. If everything fails (a getter cannot be found), we return the name of the object that contains the property. This suffices for `source_nb` to work.

In [10]:
test_eq(source_nb("Add"), 'test.ipynb')
test_eq(source_nb("Operator"), 'test.ipynb')
test_eq(source_nb("MulDiv"), 'test.ipynb')
assert source_nb(int) is None

## Reading the library

If someone decides to change a module instead of the notebooks, the following functions help update the notebooks accordingly.

In [11]:
#export
_re_cell = re.compile(r'^// Cell|^// Internal Cell|^// Comes from\s+(\S+), cell')

In [12]:
#export
def _split(code):
    lines = code.split('\n')
    nbs_path = Config().path("nbs_path").relative_to(Config().config_file.parent)
    prefix = '' if nbs_path == Path('.') else f'{nbs_path}/'
    default_nb = re.search(f'File to edit: {prefix}(\\S+)\\s+', lines[0]).groups()[0]
    s,res = 1,[]
    while _re_cell.search(lines[s]) is None: s += 1
    e = s+1
    while e < len(lines):
        while e < len(lines) and _re_cell.search(lines[e]) is None: e += 1
        grps = _re_cell.search(lines[s]).groups()
        nb = grps[0] or default_nb
        content = lines[s+1:e]
        while len(content) > 1 and content[-1] == '': content = content[:-1]
        res.append((nb, '\n'.join(content)))
        s,e = e,e+1
    return res

In [17]:
#export
def _script2notebook(fname, dic, silent=False):
    "Put the content of `fname` back in the notebooks it came from."
    if os.environ.get('IN_TEST',0): return  # don't export if running tests
    fname = Path(fname)
    with open(fname, encoding='utf8') as f: code = f.read()
    splits = _split(code)
    rel_name = fname.absolute().resolve().relative_to(Config().path("lib_path"))
    key = str(rel_name.with_suffix(''))
    assert len(splits)==len(dic[key]), f'"{rel_name}" exported from notebooks should have {len(dic[key])} cells but has {len(splits)}.'
    assert all([c1[0]==c2[1]] for c1,c2 in zip(splits, dic[key]))
    splits = [(c2[0],c1[0],c1[1]) for c1,c2 in zip(splits, dic[key])]
    nb_fnames = {Config().path("nbs_path")/s[1] for s in splits}
    for nb_fname in nb_fnames:
        nb = read_nb(nb_fname)
        for i,f,c in splits:
#             c = _deal_loc_import(c, str(fname))
            if f == nb_fname.name:
                flags = split_flags_and_code(nb['cells'][i], str)[0]
                nb['cells'][i]['source'] = flags + '\n' + c
#                 nb['cells'][i]['source'] = flags + '\n' + c.replace('', '')
        NotebookNotary().sign(nb)
        nbformat.write(nb, str(nb_fname), version=4)

    if not silent: print(f"Converted {rel_name}.")

In [18]:
#hide
dic = scala_notebook2script(silent=True, to_dict=True)
_script2notebook(Config().path("lib_path")/'test.sc', dic)

Converted test.sc.


In [15]:
#export
@call_parse
def nbdev_update_lib(fname:Param("A python filename or glob to convert", str)=None,
                     silent:Param("Don't print results", bool_arg)=False):
    "Propagates any change in the modules matching `fname` to the notebooks that created them"
#     if fname.endswith('.ipynb'): raise ValueError("`nbdev_update_lib` operates on .py files.  If you wish to convert notebooks instead, see `nbdev_build_lib`.")
    if os.environ.get('IN_TEST',0): return
    dic = scala_notebook2script(silent=True, to_dict=True)
    exported = get_nbdev_module().modules
    print(exported)

    if fname is None:
        files = [f for f in Config().path("lib_path").glob('**/*.py') if str(f.relative_to(Config().path("lib_path"))) in exported]
    else: files = glob.glob(fname)
    [ _script2notebook(f, dic, silent=silent) for f in files]

If `fname` is not specified, this will convert all modules and submodules in the `lib_folder` defined in `setting.ini`. Otherwise `fname` can be a single filename or a glob expression.

`silent` makes the command not print any statement. 

In [16]:
#hide
nbdev_update_lib()

['ModA.sc', 'ModB.sc', 'ModC.sc', 'test.sc']
